# Tansman Scheduler Project
Generic exploration and example notebook.

In [33]:
# Install required packages (need to include this for all of the tansman code)
!pip install --upgrade pip
!pip install pulp
# Doing this here rather than in config file so I don't make any assumptions about 
# the environment somebody is using in a cloned version of this project
%load_ext autoreload
%autoreload 2
# Make sure it can find modules in the project directory (always include this too)
import sys
sys.path.append('/home/nbuser/library/')

Requirement already up-to-date: pip in /home/nbuser/anaconda3_501/lib/python3.6/site-packages (20.0.2)
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Indices and Data

**Indices**
$i \in \{0, 1,  \ldots, n_{items}-1 \}$;
$t \in \{0, 1, \ldots, n_{slots}-1 \}$;
$t_\delta \in \{0, 1, \ldots, WINDOWSIZE\}$

See below for the use of $t_\delta$ and $WINDOWSIZE$ to express 
limits on practice "density".

**Data**
\begin{align*}
n_{items}             && \text{Number of items to schedule} \\
n_{slots}             && \text{Number of time slots available} \\
ENERGYUSED_i          && \text{Amount of mental "energy" required per chunk of practice time for task i} \\
TIMEAVAILABLE_t       && \text{Time available during a training slot $t$ (chunks per day)} \\
TIMEPERITEM_i         && \text{Practice time needed to master item $i$} \\
MINPERWINDOW  && \text{Minimum number of training slots for item $i$ within a window (days)} \\
MAXPERWINDOW  && \text{Maximum number of training slots for item $i$ within a window (days)} \\
WINDOWSIZE            && \text{Used to express minimum and maximum "density" of practice days (days)}\\
\end{align*}

In [66]:
# Uses pulp and my models module
import pulp
from models import *

In [37]:
sample_parameters = model_parameters(n_items=4,        # Four practice items
                       time_available=[2]*50,          # Time slots for first 50 days
                       time_per_item=[2, 2, 2, 2],     # Number of practice slots required 
                       fun_per_item=[1, 1, 2, 2],      # Fun scores (FIXME: use energy model)
                       window_size=2,                  # Window size to use
                       min_practice_per_window=0,      # Zero practice per window is acceptable
                       max_practice_per_window=1)      # But no more than 1 in 2 in window

solver = solve(**sample_parameters)
show_solution(solver)

Schedule starting tomorrow:
     name  04/01  04/02  04/03  04/04
0  item_0    0.0    1.0    0.0    1.0
1  item_1    1.0    0.0    1.0    0.0
2  item_2    0.0    1.0    0.0    1.0
3  item_3    1.0    0.0    1.0    0.0
Fun by item
[3.0, 3.0, 3.0, 3.0] Total of 12.0
Timeliness by item
[1.0, 2.0, 1.0, 2.0] Total of 6.0
Total time by item
[2.0, 2.0, 2.0, 2.0] Total of 8.0
Objective function z = 10.0
Efficiency is 8.0/8 = 1.0


In [65]:
def color_cell(val):
    bg = 'white'
    fg = 'black'
    if type(val) is not str:
        bg = "lightgray"
        if val > 0:
            bg = 'green'
            fg = 'white'
    return "background: {}; color: {}".format(bg, fg)
_df = solver.get_solution_as_df().set_index(["name"])
_df.style.applymap(color_cell)

,04/01,04/02,04/03,04/04
name,,,,
item_0,0,1,0,1
item_1,1,0,1,0
item_2,0,1,0,1
item_3,1,0,1,0
